In [28]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

# Download and convert data

In [30]:
%autoreload now

In [32]:
from document_segmentation.pagexml.annotations.generale_missiven import GeneraleMissiven
from document_segmentation.pagexml.datamodel.inventory import Inventory
from document_segmentation.settings import GENERALE_MISSIVEN_SHEET

N = None

inventories: list[Inventory] = list(
    GeneraleMissiven(GENERALE_MISSIVEN_SHEET).all_annotated_inventories(
        N, skip_errors=True
    )
)

Loading Inventories:   0%|          | 0/546 [00:00<?, ?inventory/s]

Loading Inventories:   5%|▌         | 29/546 [00:08<03:20,  2.58inventory/s]ERROR:root:Failed to load inventory 1107: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/11071.zip
ERROR:root:Failed to load inventory 1107: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/11072.zip
Loading Inventories:  13%|█▎        | 71/546 [00:27<02:52,  2.75inventory/s]ERROR:root:Scan 1001 not in inventory (Inventory(inv_nr=1212, inventory_part=, pages=918 pages))
ERROR:root:No labelled pages in inventory: Inventory(inv_nr=1212, inventory_part=, pages=918 pages)
Loading Inventories:  15%|█▍        | 80/546 [00:30<02:49,  2.75inventory/s]ERROR:root:Failed to load inventory 1234: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/12341.zip
ERROR:root:Failed to load inventory 1234: 404 Client Error: Not Found for 

# Load Data

In [5]:
%autoreload now

In [6]:
TRAINING_DATA = 0.8

In [7]:
import random

random.seed(0)
random.shuffle(inventories)

split = int(len(inventories) * TRAINING_DATA)

training_data: list[Inventory] = inventories[:split]
validation_data: list[Inventory] = inventories[split:]

In [8]:
training_data

[Inventory(inv_nr=1068, inventory_part=, pages=7 pages),
 Inventory(inv_nr=1073, inventory_part=, pages=20 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
 Inventory(inv_nr=1075, inventory_part=, pages=18 pages),
 Inventory(inv_nr=1070, inventory_part=, pages=31 pages),
 Inventory(inv_nr=1068, inventory_part=, pages=19 pages),
 Inventory(inv_nr=1071, inventory_part=, pages=21 pages),
 Inventory(inv_nr=1075, inventory_part=, pages=2 pages),
 Inventory(inv_nr=1075, inventory_part=, pages=29 pages),
 Inventory(inv_nr=1075, inventory_part=, pages=18 pages),
 Inventory(inv_nr=1079, inventory_part=, pages=98 pages),
 Inventory(inv_nr=1079, inventory_part=, pages=5 pages),
 Inventory(inv_nr=1070, inventory_part=, pages=9 pages),
 Inventory(inv_nr=1068, inventory_part=, pages=24 pages),
 Inventory(inv_nr=1076, inventory_part=, pages=48 pages),
 Inventory(inv_nr=1070, inventory_part=, pages=52 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=2 pages),
 Inventory(inv_nr=1

In [9]:
validation_data

[Inventory(inv_nr=1071, inventory_part=, pages=21 pages),
 Inventory(inv_nr=1068, inventory_part=, pages=35 pages),
 Inventory(inv_nr=1073, inventory_part=, pages=11 pages),
 Inventory(inv_nr=1079, inventory_part=, pages=13 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=3 pages),
 Inventory(inv_nr=1082, inventory_part=, pages=97 pages)]

# Train Model

In [10]:
EPOCHS = 3

In [11]:
%autoreload now

In [12]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [13]:
tagger._device

'mps'

In [14]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(10, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_featur

In [15]:
tagger.train_(training_data, validation_data, epochs=EPOCHS, shuffle=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carschno. Use `wandb login --relogin` to force relogin


Evaluating: 100%|██████████| 6/6 [00:06<00:00,  1.02s/batch]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 6/6 [00:00<00:00, 47.03batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 6/6 [00:00<00:00, 38.89batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


MulticlassAccuracy,▁██
epoch,▁▅█
inventory length,▃▃▂▂▂▁▂▁▂▃▄▁▁██▃▁▂▂▅▁▂▃▁▂▁▂▂▂▂▁▂▂▁▃▃▂▃▂▁
loss,▄▇█▄▃▆▃▄▃▂▂▃▃▁▁▁▂▃▂▁▃▂▅▂▁▂▁▂▂▁▃▁▁▂▁▁▆▅▁▂
MulticlassAccuracy,0.96111
epoch,2
inventory length,2
loss,6.96137


# Evaluate Model

In [18]:
import sys

precision, recall, f1, accuracy = tagger.eval_(validation_data, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:  33%|███▎      | 2/6 [00:00<00:00, 16.33batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_1071_0175.jpg	Ed: Erntfeste. Wijse. Voorsinnige, seer discrete; 	[0.00011291017290204763, 0.9886766672134399, 0.010757148265838623, 0.00017022430256474763, 0.00028312456561252475]
IN	IN	NL-HaNA_1.04.02_1071_0176.jpg	tegens Bantam  den; Mataram.; Sampson, Zeewolff & 	[0.00044117242214269936, 0.32972392439842224, 0.6657510995864868, 0.0034680485259741545, 0.0006157075404189527]
IN	IN	NL-HaNA_1.04.02_1071_0177.jpg	nilla geweest Ao. 11620.; goee quartiteijt pepar v	[0.00017569675401318818, 0.03183109685778618, 0.965394139289856, 0.0023813415318727493, 0.00021779445523861796]
IN	IN	NL-HaNA_1.04.02_1071_0178.jpg	jacht Setrate met nagelen; wt Matico ende Anvogno.	[7.644436846021563e-05, 0.002789461752399802, 0.9950441122055054, 0.0020284864585846663, 6.151523120934144e-05]
IN	IN	NL-HaNA_1.04.02_1071_0179.jpg	van Nagelen in Prolnco &; Amboyno.; voornemen vand	[7.25830759620294e-05, 0.0024261041544377804, 0.9951682090759277, 0.002274795901030302, 5.834133116877638e-0

Evaluating: 100%|██████████| 6/6 [00:00<00:00, 19.61batch/s]

IN	IN	NL-HaNA_1.04.02_1082_0315.jpg	ende Voort aen uijt de Schepen t'handelen, omme so	[0.0005504735163412988, 0.013193177990615368, 0.9662430286407471, 0.01957731880247593, 0.0004359275917522609]
IN	IN	NL-HaNA_1.04.02_1082_0316.jpg	1acht Clein Zeelant; in Siam gekemp; off twee gesi	[0.0005543562001548707, 0.013146469369530678, 0.9662634134292603, 0.019595397636294365, 0.00044037107727490366]
IN	IN	NL-HaNA_1.04.02_1082_0317.jpg	Heltevoll vat Siam; na Japan.; soo t' gevonden wie	[0.0005556934629566967, 0.0135355768725276, 0.9662235379219055, 0.019242649897933006, 0.0004424750222824514]
IN	IN	NL-HaNA_1.04.02_1082_0318.jpg	ster 3. Jonckx 16t; patanij; Pr. Maurits vonk; aen	[0.0005485911387950182, 0.013458743691444397, 0.9657066464424133, 0.01985275186598301, 0.0004332402313593775]
IN	IN	NL-HaNA_1.04.02_1082_0319.jpg	d'Engelsz. Beer; x sope. 8t; Scheeps-prinsien; geh	[0.0005465974099934101, 0.012963167391717434, 0.9659983515739441, 0.020060142502188683, 0.00043172395089641213]
IN	IN	NL-HaN

In [17]:
import csv

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	UNK	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	1.0000	0.9877	0.5455	0.0000
MulticlassRecall	0.0000	0.8571	0.9699	0.8571	0.0000
MulticlassF1Score	0.0000	0.9231	0.9787	0.6667	0.0000
MulticlassAccuracy (micro average):	0.9611
